In [1]:
#  Libraries
import requests
from datetime import datetime, timedelta

In [2]:
api_base = 'https://geo.weather.gc.ca/geomet'

In [3]:
# Define a function to round a number to the nearest multiple [ Api only shows requests for ]
def round_multiple(num, multiple):
    return ((num + multiple // 2) // multiple) * multiple

In [4]:
#  Function to get the API datetime
def get_api_datetime(time: datetime) -> str:
    utc_time = datetime.utcfromtimestamp(time.timestamp())
    rounded_hour = round_multiple(utc_time.hour, 3)
    if rounded_hour >= 24:
        utc_time = utc_time.replace(hour=0, minute=0, second=0) + timedelta(days=1)
    else:
        utc_time = utc_time.replace(hour=rounded_hour, minute=0, second=0)
    return utc_time.isoformat(timespec='seconds')

In [5]:
# Define a function to get the API URL
def get_api_url(time: datetime, lat: float, lng: float, layers='GDPS.ETA_TT'):
    pred_time = get_api_datetime(time)
    url = (
    'https://geo.weather.gc.ca/geomet?SERVICE=WMS&VERSION=1.3.0'
    '&REQUEST=GetFeatureInfo'
    '&BBOX={lat},{lng},{lat_end},{lng_end}'
    '&CRS=EPSG:4326&WIDTH=10&HEIGHT=10'
    '&LAYERS={layers}'
    '&QUERY_LAYERS={layers}'
    '&INFO_FORMAT=application/json'
    '&I=5&J=5'
    '&TIME={time}Z'
    )
    return url.format(lat=lat, lng=lng, lat_end=lat+0.5, lng_end=lng+0.5, time=pred_time, layers=layers), pred_time

In [6]:
# Function to get the air temperature
def get_air_temperature(time: datetime):
    url, pred_time = get_api_url(time, lat=53, lng=-113) #City of Edmonton 
    res = requests.get(url)
    return res.json(), pred_time

In [7]:
# Get the air temperature and prediction time
weather_data, pred_time = get_air_temperature(datetime.now())

# Print the prediction time
print('Prediction Time:', pred_time)

# Print the weather data
print('Weather Data:', weather_data)

# Extract and print the temperature from the weather data
temperature = weather_data['features'][0]['properties']['value']
print('Temperature:', temperature, '°C')

Prediction Time: 2023-06-26T21:00:00
Weather Data: {'type': 'FeatureCollection', 'layer': 'GDPS.ETA_TT', 'features': [{'type': 'Feature', 'id': 'GDPS.ETA_TT/-112.8/53.25', 'geometry': {'type': 'Point', 'coordinates': [-112.8, 53.25]}, 'properties': {'value': '23.866693', 'class': '20 25'}}]}
Temperature: 23.866693 °C


In [8]:
import pandas as pd
from bokeh.plotting import show, figure, output_notebook

In [9]:
# Define a function to get air temperature predictions
def get_air_temp_predictions(num=5) -> pd.DataFrame:
    air_temps = []
    pred_times = []
    now = datetime.now()
    times = [now + timedelta(hours=3 * i) for i in range(num)]
    for time in times:
        air_temp_json, pred_time_str = get_air_temperature(time)
        air_temp = air_temp_json['features'][0]['properties']['value']
        air_temps.append(float(air_temp))
        pred_times.append(pred_time_str)
    df = pd.DataFrame({'air_temp': air_temps}, index=pred_times)
    df.index = pd.to_datetime(df.index)
    return df


In [10]:
# Get air temperature predictions
air_temps_df = get_air_temp_predictions()

In [11]:
# Display the dataframe
air_temps_df

,air_temp
2023-06-26 21:00:00,23.866693
2023-06-27 00:00:00,24.483789
2023-06-27 03:00:00,22.601740
2023-06-27 06:00:00,17.610926
2023-06-27 09:00:00,14.901789


In [12]:
# Define a function to plot the weather data
def plot_weather_data(df):
    output_notebook()
    x = df.index
    y = df['air_temp']
    y_min = round_multiple(y.min(), 10)
    if y_min > y.min():
        y_min -= 10
    y_max = round_multiple(y.max(), 10)
    if y_max < y.max():
        y_max += 10
    p = figure(title='Air temperature forecast', x_axis_type='datetime',
               x_axis_label='Time of day', y_axis_label='Temperature (Celsius)',
               y_range=[y_min, y_max])
    p.line(x=x, y=y)
    p.scatter(x=x, y=y)
    return p

# Plot the weather data
p = plot_weather_data(air_temps_df)
show(p)

Loading BokehJS ...

In [13]:
import ipywidgets as widgets
from IPython.display import clear_output

# Define a function to fetch and plot the data
def fetch_and_plot():
    # Get air temperature predictions
    air_temps_df = get_air_temp_predictions()

    # Plot the weather data
    p = plot_weather_data(air_temps_df)
    return p

# Define a function to be called when the button is pressed
def update(button):
    # Fetch and plot the new data
    new_p = fetch_and_plot()

    # Clear the old output and display the new plot
    clear_output(wait=True)
    output_notebook()
    show(new_p, notebook_handle=True)

# Create a button widget
button = widgets.Button(description="Refresh Data")

# Set the button's callback to the update function
button.on_click(update)

# Fetch and plot the initial data
initial_p = fetch_and_plot()

# Display the initial plot and the button
output_notebook()
show(initial_p, notebook_handle=True)
display(button)

Loading BokehJS ...

Loading BokehJS ...

Button(description='Refresh Data', style=ButtonStyle())